In [1]:
# 1. create a Policy Network & Value Network
# Policy: MLP with (State -> Act)
# Value: MLP with (State -> 1(value))

# 2. VPGD algorithm taking Policy, Value, Envornment as input


# 3. plot loss

# *4. update to hugging face


In [18]:
!pip install -r https://raw.githubusercontent.com/huggingface/deep-rl-class/main/notebooks/unit4/requirements-unit4.txt
!pip install pygame ple

  Cloning https://github.com/ntasfi/PyGame-Learning-Environment.git to /private/var/folders/_t/z3tjm9014td81ypk6h_0423c0000gn/T/pip-req-build-1arjw3cc
  Running command git clone --filter=blob:none --quiet https://github.com/ntasfi/PyGame-Learning-Environment.git /private/var/folders/_t/z3tjm9014td81ypk6h_0423c0000gn/T/pip-req-build-1arjw3cc
  Resolved https://github.com/ntasfi/PyGame-Learning-Environment.git to commit 3dbe79dc0c35559bb441b9359948aabf9bb3d331
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/simoninithomas/gym-games to /private/var/folders/_t/z3tjm9014td81ypk6h_0423c0000gn/T/pip-req-build-qpdd_naz
  Running command git clone --filter=blob:none --quiet https://github.com/simoninithomas/gym-games /private/var/folders/_t/z3tjm9014td81ypk6h_0423c0000gn/T/pip-req-build-qpdd_naz
  Resolved https://github.com/simoninithomas/gym-games to commit f31695e4ba028400628dc054ee8a436f28193f0b
  Preparing metadata (setup.py) ... done


In [31]:
from huggingface_hub import HfApi, snapshot_download
from huggingface_hub.repocard import metadata_eval_result, metadata_save

from pathlib import Path
import datetime
import json
import imageio

import tempfile

import os

def record_video(env, policy, out_directory, fps=30):
    """
    Generate a replay video of the agent
    :param env
    :param Qtable: Qtable of our agent
    :param out_directory
    :param fps: how many frame per seconds (with taxi-v3 and frozenlake-v1 we use 1)
    """
    images = []
    done = False
    state = env.reset()
    img = env.render(mode="rgb_array")
    images.append(img)
    while not done:
        # Take the action (index) that have the maximum expected future reward given that state
        action = take_action(policy, torch.as_tensor(state))
        state, reward, done, info = env.step(action)  # We directly put next_state = state for recording logic
        img = env.render(mode="rgb_array")
        images.append(img)
    imageio.mimsave(out_directory, [np.array(img) for i, img in enumerate(images)], fps=fps)
    
def evaluate_agent(env, max_steps, n_eval_episodes, policy):
    """
    Evaluate the agent for ``n_eval_episodes`` episodes and returns average reward and std of reward.
    :param env: The evaluation environment
    :param n_eval_episodes: Number of episode to evaluate the agent
    :param policy: The Reinforce agent
    """
    episode_rewards = []
    for episode in range(n_eval_episodes):
        state = env.reset()
        step = 0
        done = False
        total_rewards_ep = 0

        for step in range(max_steps):
            action, _ = policy.act(state)
            new_state, reward, done, info = env.step(action)
            total_rewards_ep += reward

            if done:
                break
            state = new_state
        episode_rewards.append(total_rewards_ep)
    mean_reward = np.mean(episode_rewards)
    std_reward = np.std(episode_rewards)

    return mean_reward, std_reward

def push_to_hub(repo_id,
                model,
                hyperparameters,
                eval_env,
                video_fps=30
                ):
  """
  Evaluate, Generate a video and Upload a model to Hugging Face Hub.
  This method does the complete pipeline:
  - It evaluates the model
  - It generates the model card
  - It generates a replay video of the agent
  - It pushes everything to the Hub

  :param repo_id: repo_id: id of the model repository from the Hugging Face Hub
  :param model: the pytorch model we want to save
  :param hyperparameters: training hyperparameters
  :param eval_env: evaluation environment
  :param video_fps: how many frame per seconds to record our video replay
  """

  _, repo_name = repo_id.split("/")
  api = HfApi()

  # Step 1: Create the repo
  repo_url = api.create_repo(
        repo_id=repo_id,
        exist_ok=True,
  )

  with tempfile.TemporaryDirectory() as tmpdirname:
    local_directory = Path(tmpdirname)

    # Step 2: Save the model
    torch.save(model, local_directory / "model.pt")

    # Step 3: Save the hyperparameters to JSON
    with open(local_directory / "hyperparameters.json", "w") as outfile:
      json.dump(hyperparameters, outfile)

    # Step 4: Evaluate the model and build JSON
    mean_reward, std_reward = evaluate_agent(eval_env,
                                            hyperparameters["max_t"],
                                            hyperparameters["n_evaluation_episodes"],
                                            model)
    # Get datetime
    eval_datetime = datetime.datetime.now()
    eval_form_datetime = eval_datetime.isoformat()

    evaluate_data = {
          "env_id": hyperparameters["env_id"],
          "mean_reward": mean_reward,
          "n_evaluation_episodes": hyperparameters["n_evaluation_episodes"],
          "eval_datetime": eval_form_datetime,
    }

    # Write a JSON file
    with open(local_directory / "results.json", "w") as outfile:
        json.dump(evaluate_data, outfile)

    # Step 5: Create the model card
    env_name = hyperparameters["env_id"]

    metadata = {}
    metadata["tags"] = [
          env_name,
          "reinforce",
          "reinforcement-learning",
          "custom-implementation",
          "deep-rl-class"
      ]

    # Add metrics
    eval = metadata_eval_result(
        model_pretty_name=repo_name,
        task_pretty_name="reinforcement-learning",
        task_id="reinforcement-learning",
        metrics_pretty_name="mean_reward",
        metrics_id="mean_reward",
        metrics_value=f"{mean_reward:.2f} +/- {std_reward:.2f}",
        dataset_pretty_name=env_name,
        dataset_id=env_name,
      )

    # Merges both dictionaries
    metadata = {**metadata, **eval}

    model_card = f"""
  # **Reinforce** Agent playing **{env_id}**
  This is a trained model of a **Reinforce** agent playing **{env_id}** .
  To learn to use this model and train yours check Unit 4 of the Deep Reinforcement Learning Course: https://huggingface.co/deep-rl-course/unit4/introduction
  """

    readme_path = local_directory / "README.md"
    readme = ""
    if readme_path.exists():
        with readme_path.open("r", encoding="utf8") as f:
          readme = f.read()
    else:
      readme = model_card

    with readme_path.open("w", encoding="utf-8") as f:
      f.write(readme)

    # Save our metrics to Readme metadata
    metadata_save(readme_path, metadata)

    # Step 6: Record a video
    video_path =  local_directory / "replay.mp4"
    record_video(env, model, video_path, video_fps)

    # Step 7. Push everything to the Hub
    api.upload_folder(
          repo_id=repo_id,
          folder_path=local_directory,
          path_in_repo=".",
    )

    print(f"Your model is pushed to the Hub. You can view your model here: {repo_url}")

# TODO
1. check performance
2. check execution time

In [ ]:
# vanila policy gradient descent with reward to go and BASELINE
import torch
import torch.nn as nn
import torch.optim as optim
import gym
import gym_pygame
import numpy as np
from timeit import default_timer as timer
import matplotlib.pyplot as plt
import random
import os
from datetime import datetime

# hyperparamebers to tune
learning_rate =  1e-3
gamma = 0.9

debug = False

# File path to the model
policy_model_path = "policy_model.pth"
value_model_path = "value_model.pth"
video_fps = 30
# video_path = 'replay.mp4'


def load_model(model, model_file_path):
  # Check if the model file exists
    if os.path.exists(model_file_path):
        print(f"Loading model from {model_file_path}...")
        # Load the saved model state
        model.load_state_dict(torch.load(model_file_path))
    else:
        print(f"No saved model found at {model_file_path}. Creating a new model...")
        
def save_model(model, model_file_path):
    # Save model
    torch.save(model.state_dict(), model_file_path)

def disable_random():
    # disable all random ness
    # Set the random seed for Python's random module
    random.seed(0)
    # Set the random seed for NumPy (if you're using it)
    np.random.seed(0)
    # Set the random seed for PyTorch (CPU and GPU)
    torch.manual_seed(0)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(0)

def print_trajectories(batch_traj):
    #debug if action is random enough
    if debug:
        actions = []
        for st, at, reward, st_1, done in batch_traj:
            actions.append(at)
        print(f'actions: {actions}')

def print_rewards(traj_rewards):
    #debug if the random b actually yield difference in rewards
    if debug:
        print(f'traj rewards: {traj_rewards}')

class Policy(nn.Module):
    # (TODO) MLP/Policy can be reconfigured to contain multiple layers
    def __init__(self, dim_input, dim_hidden, dim_output):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(dim_input, dim_hidden),
            nn.ReLU(),
            nn.Linear(dim_hidden, 2 * dim_hidden),
            nn.ReLU(),
            nn.Linear(2 * dim_hidden, dim_hidden),
            nn.ReLU(),
            nn.Linear(dim_hidden, dim_output)
        )
    # return logits of size(dim_output)
    def forward(self, x):
        return self.mlp(x)
    
class Value(nn.Module):
    # (TODO) MLP/Policy can be reconfigured to contain multiple layers
    def __init__(self, dim_input, dim_hidden):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(dim_input, dim_hidden),
            nn.ReLU(),
            nn.Linear(dim_hidden, 2 * dim_hidden),
            nn.ReLU(),
            nn.Linear(2 * dim_hidden, dim_hidden),
            nn.ReLU(),
            nn.Linear(dim_hidden, 1)
        )
        
    def forward(self, x):
        return self.mlp(x)


# action(st), H(st) -> at based on probability distribution
def take_action(policy, s):
    logits = policy(s)
    probs = torch.softmax(logits, dim=-1)
    categorical_dist = torch.distributions.Categorical(probs = probs)
    action = categorical_dist.sample()
    return action.item()

def get_logp(policy, obs, acts):
    logits = policy(obs)
    categorical_dists = torch.distributions.Categorical(logits = logits)
    return categorical_dists.log_prob(acts)

def get_values(value, obs):
    baselines = value(obs)
    return baselines

    
# return loss, avg_reward, avg_lens for this epoch
def train_epoch(env, policy, value, num_batch, policy_opt, value_opt):
    # return [(st, at, reward, st+1, done)]
    def generate_trajectory(env, policy, num_batch):
        batch_traj = []
        for i in range(num_batch):
            st = env.reset()
            done = False
            while not done:
                action = take_action(policy, torch.tensor(st))
                st_1, reward, done, _ = env.step(action)
                batch_traj.append((st, action, reward, st_1, done))
                st = st_1
        return batch_traj
    
    # return reward to go
    def compute_reward_to_go(batch_traj):
        rtgs = []
        curr_traj = []
        for st, at, reward, st_1, done in batch_traj:
            curr_traj.append((st, at, reward, st_1, done))
            if done:
                # append rewards to gos from current trajectory to rtgs
                rtgs.extend(get_rtgs_from_current_traj(curr_traj))
                curr_traj = []
        return rtgs
                
    def get_rtgs_from_current_traj(curr_traj):
        n = len(curr_traj)
        rtgs = [0] * n
        for i in reversed(range(n)):
            st, at, reward, st_1, done = curr_traj[i]
            if i == n - 1:
                rtgs[i] = reward
            else:
                rtgs[i] = reward + gamma * rtgs[i + 1]
        return rtgs 
    
    # traj: [(st, at, reward, st+1, done)]
    def fit_value_evaluation(value, value_opt, batch_traj):
        predications = []
        targets = []
        for st, at, reward, st_1, done in batch_traj:
            v_st = value(torch.as_tensor(st))
            v_st_1 = value(torch.as_tensor(st_1))
            predications.append(v_st)
            targets.append(reward + gamma * v_st_1)
        value_opt.zero_grad()
        targets = torch.as_tensor(targets, dtype=torch.float64)
        predications = torch.stack(predications).squeeze(-1)
        loss = nn.MSELoss()(predications, targets)
        loss.backward()
        value_opt.step()
        return loss.item()
        
    
    def fit_policy_evaluation(policy, policy_opt, batch_traj, value):
        batch_adv = [] # [tensor(torch.double)]
        batch_st = []
        batch_at = []
        # compute adv(st, at)
        with torch.no_grad():
            for st, at, reward, st_1, done in batch_traj:
                v_st = value(torch.as_tensor(st)).item()
                v_st_1 = value(torch.as_tensor(st_1)).item()
                batch_adv.append(reward + gamma * v_st_1 - v_st)
                batch_st.append(st)
                batch_at.append(at)
        batch_adv = torch.as_tensor(batch_adv)
        logps = get_logp(policy = policy, obs= torch.as_tensor(batch_st), acts = torch.as_tensor(batch_at))
        loss = -(logps * batch_adv).mean()
        policy_opt.zero_grad()
        loss.backward()
        policy_opt.step()
        return loss.item()
    
    # return mean(reward), len(reward), std(reward)
    def evaluate_policy(batch_traj):
        sampled_rewards = []
        traj_lens = []
        traj_reward = 0
        len = 0
        for st, at, reward, st_1, done in batch_traj:
            len += 1
            traj_reward += reward
            if done:
                sampled_rewards.append(traj_reward)
                traj_lens.append(len)
                traj_reward = 0
                len = 0
        return np.mean(sampled_rewards), np.std(sampled_rewards), np.mean(traj_lens)
                
        
    # start = timer()
    batch_traj = generate_trajectory(env, policy, num_batch)
    # batch_rtgs = compute_reward_to_go(batch_traj)
    print_trajectories(batch_traj)
    # print_rewards(batch_rtgs)
    v_loss = fit_value_evaluation(value = value, value_opt=value_opt, batch_traj=batch_traj)
    p_loss = fit_policy_evaluation(policy = policy, policy_opt = policy_opt, batch_traj = batch_traj, value = value)
    reward_mean, reward_std, len_mean = evaluate_policy(batch_traj)
    
    return p_loss, v_loss, reward_mean, len_mean, reward_std
def train():
    start = timer()
    #debug
    # disable_random(
    env = gym.make("Pixelcopter-PLE-v0")
    # make the action deterministic
    dim_input = env.observation_space.shape[0]
    dim_output = env.action_space.n
    policy = Policy(dim_input, 32, dim_output).double()
    value = Value(dim_input, 32).double()
    # load_model(policy, policy_model_path)
    # load_model(value, value_model_path)
    policy_opt = torch.optim.Adam(policy.parameters(), lr = learning_rate)
    value_opt = torch.optim.Adam(value.parameters(), lr = learning_rate)

    num_epoch = 50
    num_batch = 100
    rewards = []
    reward_std = []
    for i in range(num_epoch):
        p_loss, v_loss, reward, lens, reward_st = train_epoch(env, policy, value, num_batch, policy_opt, value_opt)
        if i % 10 == 0:
            print(f"{i}th iteration, p loss={p_loss}, v_loss={v_loss} rewards={reward}, std={reward_st} lens={lens}\n")
        # print(f"{i}th iteration, loss={loss}, rewards={reward}, std={reward_st} lens={lens}\n")

        rewards.append(reward)
        reward_std.append(reward_st)
    print(f'total time for traning: {timer() - start}')
    plt.plot(list(range(num_epoch)), rewards, marker='o', linestyle='-', label='Rewards per Epoch')
    plt.plot(list(range(num_epoch)), reward_std, marker='s', linestyle='--',  color='red', label='std per Epoch')
    plt.xlabel('Epochs')
    plt.ylabel('Rewards')
    plt.title('Rewards per Epoch')
    plt.legend()
    plt.grid(True)
    plt.show()
    save_model(policy, policy_model_path)
    save_model(value, value_model_path)
    policy.eval()
    current_time = datetime.now()

    # Format the date and time as mm-dd-tt
    formatted_time = current_time.strftime("%m-%d-%H:%M")  # %H%M represents hours and minutes (24-hour format)

    # Append the formatted time to the string 'replay'
    video_path = f"replay-{formatted_time}.mp4"
    # record_video(env, policy, video_path, video_fps)

train()





Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
couldn't import doomish
Couldn't import doom


/Users/liqiu/opt/anaconda3/lib/python3.9/site-packages/gym/utils/passive_env_checker.py:174: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.warn(
/Users/liqiu/opt/anaconda3/lib/python3.9/site-packages/gym/utils/passive_env_checker.py:187: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.warn(
/Users/liqiu/opt/anaconda3/lib/python3.9/site-packages/gym/utils/passive_env_checker.py:195: UserWarning: WARN: The result returned by `env.reset()` was not a tuple of the form `(obs, info)`, where `obs` is a observation and `info` is a dictionary containing additional information. Actual type: `<class 'numpy.ndarray'>`
  logger.warn(
/Users/liqiu/opt/anaconda3/lib/python3.9/site-packages/gym/utils/passive_env_checker.py:219: Deprecati

0th iteration, p loss=-0.5539915035501888, v_loss=4.323989354147111 rewards=-4.94, std=0.2374868417407583 lens=5.9

10th iteration, p loss=0.013744567513046107, v_loss=1.0579919229158234 rewards=-2.01, std=1.41063815346105 lens=24.66

20th iteration, p loss=0.007731903371721825, v_loss=1.1494467297262074 rewards=-2.42, std=0.7769169839822013 lens=22.37

